<a href="https://colab.research.google.com/github/Alice1304/Section4_project/blob/main/%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B84-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 캐글에서 데이터 가져오기

- 용량이 크기 때문에 따로 다운로드 하지 않고 캐글에서 바로 데이터를 불러오기로 함

In [71]:
# 나의 캐글 Account에서 API Token 다운 받기 = > JSON 파일 다운로드 후 캐글과 연결해주기

!pip install kaggle
from google.colab import files
files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


{}

In [72]:
#Kaggle API를 사용하기 위해 json 파일을 ~/.kaggle로 이동

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning 방지
!chmod 600 ~/.kaggle/kaggle.json

In [73]:
!kaggle competitions download -c word2vec-nlp-tutorial

word2vec-nlp-tutorial.zip: Skipping, found more recently modified local copy (use --force to force download)


In [74]:
!ls

kaggle.json		  sampleSubmission.csv	  unlabeledTrainData.tsv.zip
labeledTrainData.tsv	  testData.tsv		  word2vec-nlp-tutorial.zip
labeledTrainData.tsv.zip  testData.tsv.zip
sample_data		  unlabeledTrainData.tsv


In [75]:
!unzip /content/word2vec-nlp-tutorial.zip

Archive:  /content/word2vec-nlp-tutorial.zip
replace labeledTrainData.tsv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [76]:
!unzip /content/labeledTrainData.tsv.zip

Archive:  /content/labeledTrainData.tsv.zip
replace labeledTrainData.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [77]:
!unzip /content/testData.tsv.zip

Archive:  /content/testData.tsv.zip
replace testData.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [78]:
!unzip /content/unlabeledTrainData.tsv.zip

Archive:  /content/unlabeledTrainData.tsv.zip
replace unlabeledTrainData.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [79]:
#데이터 불러오기 

In [80]:
import pandas as pd
import numpy as np
import warnings
import os
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
SEED = 33

In [81]:
train = pd.read_csv('/content/labeledTrainData.tsv', delimiter='\t')
test = pd.read_csv('/content/testData.tsv', delimiter='\t')

In [82]:
unlabeled_train = pd.read_csv('/content/unlabeledTrainData.tsv', delimiter='\t', error_bad_lines=False)

b'Skipping line 43043: expected 2 fields, saw 3\n'


In [83]:
train.shape, test.shape, unlabeled_train.shape

((25000, 3), (25000, 2), (49998, 2))

In [84]:
 unlabeled_train.head()

,id,review
0,9999_0,"Watching Time Chasers, it obvious that it was ..."
1,45057_0,I saw this film about 20 years ago and remembe...
2,15561_0,"Minor Spoilers<br /><br />In New York, Joan Ba..."
3,7161_0,I went to see this film with a great deal of e...
4,43971_0,"Yes, I agree with everyone on this site this m..."


## 2. 전처리

In [85]:
#html텍스트가 포함되어 있어 삭제
#불용어 제거

from bs4 import BeautifulSoup
import re

In [86]:
 soup = BeautifulSoup()

In [87]:
sample = train['review'][0]

In [88]:
sample

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [89]:
soup = BeautifulSoup(sample, 'html.parser')

In [90]:
soup.text  #html삭제

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 20 mi

In [91]:
clean = re.sub('[^a-zA-Z]', ' ', soup.text)  #알파벳을 제외한 문자 모두 공백처리

In [92]:
clean

'With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    mi

In [93]:
clean.lower()

'with all this stuff going down at the moment with mj i ve started listening to his music  watching the odd documentary here and there  watched the wiz and watched moonwalker again  maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  some of it has subtle messages about mj s feeling towards the press and also the obvious message of drugs are bad m kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring  some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him the actual feature film bit when it finally starts is only on for    mi

In [94]:
# 위와 같은 과정을 하나의 함수로 만들어주기

def preprocessing(sentence):
  soup = BeautifulSoup(sentence, 'html.parser')
  clean = re.sub('[^a-zA-Z]', ' ', soup.text)
  clean.lower()

In [95]:
!pip install nltk
import nltk
nltk.download("stopwords")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [96]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each